In [1]:
import sqlite3
import numpy as np
from SlideRunner_dataAccess.database import Database
from tqdm import tqdm
from pathlib import Path
import openslide
import time
import pickle
import cv2
import torchvision.transforms as transforms
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
import sys

sys.path.append('lib/')
from data_loader import *

from lib.object_detection_helper import *
from model.RetinaNetFocalLoss import RetinaNetFocalLoss
from model.RetinaNet import RetinaNet
import sys
import threading
from queue import Queue
import queue


Done importing database


/localhome/aubrevillelocal/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/localhome/aubrevillelocal/SlideRunner/SlideRunner/dataAccess/database.py:2: UserWarning: Deprecated package. Please use the SlideRunner_dataAccess package, and not the SlideRunner main package for data access.
  warnings.warn('Deprecated package. Please use the SlideRunner_dataAccess package, and not the SlideRunner main package for data access.')


In [2]:
size=512
path = Path('./')

database = Database()
database.open('MIDOG.sqlite')
slidedir = 'images_training'

size = 512
level = 0

test_files = []


test_slide_filenames = np.arange(41,51).tolist()+np.arange(91,101).tolist()+np.arange(141,151).tolist()

files=list()
train_files={'XR':[], 'S360':[],'CS2':[]}
slidenames = list()
getslides = """SELECT uid, directory, filename FROM Slides"""
for idx, (currslide, folder, filename) in enumerate(tqdm(database.execute(getslides).fetchall(), desc='Loading slides .. ')):
        slidenames += [currslide]

        database.loadIntoMemory(currslide)

        slide_path = path / slidedir / filename
        scont = SlideContainer(file=slide_path, level=level, width=size, height=size, y=[[], []], annotations=dict())
        if ( (currslide in test_slide_filenames)):
            test_files.append(scont)
        elif (currslide<50):
            train_files['XR'].append(scont)
        elif (currslide<100):
            train_files['S360'].append(scont)
        elif (currslide<150):
            train_files['CS2'].append(scont)
        
print('Running on slides:', slidenames)

anchors = create_anchors(sizes=[(32,32)], ratios=[1], scales=[0.6, 0.7,0.8,0.9])

detect_thresh = 0.3 
nms_thresh = 0.4
result_regression = {}




jobQueue=Queue()
outputQueue=Queue()

def getPatchesFromQueue(jobQueue, outputQueue):
    x,y=0,0
    try:
        while (True):
            if (outputQueue.qsize()<100):
                status, x,y, slide_container = jobQueue.get(timeout=60)
                if (status==-1):
                    return
                outputQueue.put((x,y,slide_container.get_patch(x, y) / 255.))
            else:
                time.sleep(0.1)
    except queue.Empty:
        print('One worker died.')
        pass # Timeout happened, exit



def getBatchFromQueue(batchsize=8):
    images = np.zeros((batchsize,3, size,size))
    x = np.zeros(batchsize)
    y = np.zeros(batchsize)
    try:
        bs=0
        for k in range(batchsize):
            x[k],y[k],images_temp = outputQueue.get(timeout=5)
            images[k] = images_temp.transpose((2,0,1))
            bs+=1
        return images,x,y
    except queue.Empty:
        return images[0:bs],x[0:bs],y[0:bs]


def rescale_box(bboxes, size: Tensor):
    bboxes[:, :2] = bboxes[:, :2] - bboxes[:, 2:] / 2
    bboxes[:, :2] = (bboxes[:, :2] + 1) * size / 2
    bboxes[:, 2:] = bboxes[:, 2:] * size / 2
    bboxes = bboxes.long()
    return bboxes


batchsize=8

# Set up queued image retrieval
jobs = []
for i in range(1):
    p = threading.Thread(target=getPatchesFromQueue, args=(jobQueue, outputQueue), daemon=True)
    jobs.append(p)
    p.start()


        










Loading slides .. : 100%|██████████| 200/200 [00:00<00:00, 292.39it/s]

Running on slides: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200]


In [3]:
len(test_files),[len(x) for x in train_files.values()]

(30, [40, 40, 40])

In [4]:
def inference(files, mean, std, result_boxes={}):
    with torch.no_grad():
        for slide_container in tqdm(files):

            size = 512

            if '/'.join(str(slide_container.file).split('/')[-1:]) in result_boxes:
                continue
            result_boxes[str(slide_container.file).split(os.sep)[-1]] = []

            n_Images=0
            for x in range(0, slide_container.slide.level_dimensions[level][0] - 1 * size, int(0.9*size)):
                for y in range(0, slide_container.slide.level_dimensions[level][1] - 1*  size, int(0.9*size)):
                    jobQueue.put((0,x,y, slide_container))
                    n_Images+=1


            for kImage in range(int(np.ceil(n_Images/batchsize))):


                    npBatch,xBatch,yBatch = getBatchFromQueue(batchsize=batchsize)
                    imageBatch = torch.from_numpy(npBatch.astype(np.float32, copy=False)).cuda()

                    patch = imageBatch

                    for p in range(patch.shape[0]):
                        patch[p] = transforms.Normalize(mean,std)(patch[p])

                    class_pred_batch, bbox_pred_batch, _ = model(
                        patch[:, :, :, :])

                    for b in range(patch.shape[0]):
                        x_real = xBatch[b]
                        y_real = yBatch[b]

                        for clas_pred, bbox_pred in zip(class_pred_batch[b][None,:,:], bbox_pred_batch[b][None,:,:],
                                                                                ):
                            modelOutput = process_output(clas_pred, bbox_pred, anchors, detect_thresh)
                            bbox_pred, scores, preds = [modelOutput[x] for x in ['bbox_pred', 'scores', 'preds']]

                            if bbox_pred is not None:
                                to_keep = nms(bbox_pred, scores, nms_thresh)
                                bbox_pred, preds, scores = bbox_pred[to_keep].cpu(), preds[to_keep].cpu(), scores[to_keep].cpu()

                                t_sz = torch.Tensor([size, size])[None].float()

                                bbox_pred = rescale_box(bbox_pred, t_sz)

                                for box, pred, score in zip(bbox_pred, preds, scores):
                                    y_box, x_box = box[:2]
                                    h, w = box[2:4]

                                    result_boxes[str(slide_container.file).split(os.sep)[-1]].append(np.array([x_box + x_real, y_box + y_real,
                                                                                             x_box + x_real + w, y_box + y_real + h,
                                                                                             pred, score]))
    return result_boxes

In [5]:
def load_model(fname):
    state = torch.load(fname, map_location='cpu')     if defaults.device == torch.device('cpu')     else torch.load(fname)
    model = state.pop('model').cuda()
    mean = state['data']['normalize']['mean']
    std = state['data']['normalize']['std']
    device = torch.device("cuda:0")
    model = model.cuda(device)
    return model, mean, std

In [6]:
result_boxes_train = {'S360':{},'CS2':{},'XR':{}}

for scanner in ['S360','CS2','XR']:
    for run in np.arange(1,6):
        fname = f'RetinaNet-MIDOG-{scanner}-{run}.pth'
        model,mean,std = load_model(fname)
        result_boxes_train[scanner][run] = inference(train_files[scanner],mean,std,result_boxes={})
        

100%|██████████| 40/40 [04:17<00:00,  6.47s/it]


In [7]:
database = Database()
database.open('TUPAC_AL/TUPAC_alternativeLabels_training.sqlite')
slidedir = 'TUPAC_AL/TUPACstitched/'

size = 512
level = 0

train_files['TUPAC'] = []
getslides = """SELECT uid, directory, filename FROM Slides"""
for idx, (currslide, folder, filename) in enumerate(tqdm(database.execute(getslides).fetchall(), desc='Loading slides .. ')):
        slidenames += [currslide]

        database.loadIntoMemory(currslide)

        slide_path = path / slidedir / filename
        scont = SlideContainer(file=slide_path, level=level, width=size, height=size, y=[[], []], annotations=dict())
        train_files['TUPAC'].append(scont)

Loading slides .. : 100%|██████████| 73/73 [00:00<00:00, 160.37it/s]


In [8]:
result_boxes_train['TUPAC']={}
for run in np.arange(1,6):
    fname = f'RetinaNet-TUPAC_AL-OrigSplit-512s-run{run}.pth'
    model,mean,std = load_model(fname)
    result_boxes_train['TUPAC'][run] = inference(train_files['TUPAC'],mean=mean,std=std,result_boxes={})
    

100%|██████████| 73/73 [08:51<00:00,  1.89s/it]


In [9]:
result_boxes = {'TUPAC':{},'S360':{},'CS2':{},'XR':{}}
for run in np.arange(1,6):
    fname = f'RetinaNet-TUPAC_AL-OrigSplit-512s-run{run}.pth'
    model,mean,std = load_model(fname)
    result_boxes['TUPAC'][run] = inference(test_files,mean=mean,std=std,result_boxes={})

100%|██████████| 30/30 [03:09<00:00,  6.16s/it]


In [10]:
for scanner in ['S360','CS2','XR']:
    for run in np.arange(1,6):
        fname = f'RetinaNet-MIDOG-{scanner}-{run}.pth'
        model,mean,std = load_model(fname)
        result_boxes[scanner][run] = inference(test_files,mean,std,result_boxes={})

100%|██████████| 30/30 [03:09<00:00,  6.13s/it]


In [11]:
from lib.nms_WSI import nms as nms_WSI
from lib.calculate_F1 import _F1_core

def calculate_F1(DB, result_boxes=None, det_thres=0.5, hotclass=1,verbose=False):

    if (result_boxes is None):
        if resfile is None:
            raise ValueError('At least one of resfile/result_boxes must be given')
    
    sTP, sFN, sFP = 0,0,0
    F1dict = dict()
    sP = 0
    
    result_boxes = nms_WSI(result_boxes, det_thres)
    
#    print('Calculating F1 for test set of %d files' % len(result_boxes),':',result_boxes.keys())
    
    slideids = []
    
    for resfile in result_boxes:
        boxes = np.array(result_boxes[resfile])
        

        TP, FP, FN,F1 = 0,0,0,0
        slide_id=DB.findSlideWithFilename(resfile,'')
        slideids.append(str(slide_id))
        DB.loadIntoMemory(slide_id)

        annoList=[]
        for annoI in DB.annotations:
            anno = DB.annotations[annoI]
            if anno.agreedClass==hotclass:
                annoList.append([anno.x1,anno.y1])

        centers_DB = np.array(annoList)

        if boxes.shape[0]>0:
            score = boxes[:,-1]
            
            F1,TP,FP,FN = _F1_core(centers_DB, boxes, score,det_thres)
            if (centers_DB.shape[0] != TP+FN):
                print(resfile,centers_DB.shape[0],TP+FN)
        else: # no detections --> missed all
            FN = centers_DB.shape[0] 
        
        if (verbose):
            print(f'{resfile}: F1:{F1}, TP:{TP}, FP:{FP}, FN:{FN}')


        sTP+=TP
        sFP+=FP
        sP += centers_DB.shape[0]
        sFN+=FN
        F1dict[resfile]=F1
        
    sF1 = 2*sTP/(2*sTP + sFP + sFN)
    #print('F1: ',sF1)
    #print('Detections:', sFP+sTP)
    #print('Precision: %.3f '%(sTP / (sTP+sFP)))
    #print('Recall: %.3f' %(sTP / (sTP+sFN)))
    
    return sF1


In [12]:
def optimize_threshold(DB, result_boxes=None, hotclass=1, minthres=0.3):


    sTP, sFN, sFP = 0,0,0
    F1dict = dict()
    
    MIN_THR = minthres

    result_boxes = nms_WSI(result_boxes, MIN_THR)
    TPd, FPd, FNd, F1d = dict(), dict(), dict(), dict()
    thresholds = np.arange(MIN_THR,0.99,0.01)
    
    print('Optimizing threshold for validation set of %d files: '%len(result_boxes.keys()))

    for resfile in result_boxes:
        boxes = np.array(result_boxes[resfile])

        TP, FP, FN = 0,0,0
        TPd[resfile] = list()
        FPd[resfile] = list()
        FNd[resfile] = list()
        F1d[resfile] = list()

        if (boxes.shape[0]>0):
            score = boxes[:,-1]

            DB.loadIntoMemory(DB.findSlideWithFilename(resfile,''))
        
            # perform NMS on detections

            annoList=[]
            for annoI in DB.annotations:
                anno = DB.annotations[annoI]
                if anno.agreedClass==hotclass:
                    annoList.append([anno.x1,anno.y1])

            centers_DB = np.array(annoList)



            for det_thres in thresholds:
                F1,TP,FP,FN = _F1_core(centers_DB, boxes, score,det_thres)
                TPd[resfile] += [TP]
                FPd[resfile] += [FP]
                FNd[resfile] += [FN]
                F1d[resfile] += [F1]
        else:
            for det_thres in thresholds:
                TPd[resfile] += [0]
                FPd[resfile] += [0]
                FNd[resfile] += [0]
                F1d[resfile] += [0]
            F1 = 0
            

        F1dict[resfile]=F1

    allTP = np.zeros(len(thresholds))
    allFP = np.zeros(len(thresholds))
    allFN = np.zeros(len(thresholds))
    allF1 = np.zeros(len(thresholds))
    allF1M = np.zeros(len(thresholds))



    for k in range(len(thresholds)):
        allTP[k] = np.sum([TPd[x][k] for x in result_boxes])
        allFP[k] = np.sum([FPd[x][k] for x in result_boxes])
        allFN[k] = np.sum([FNd[x][k] for x in result_boxes])
        allF1[k] = 2*allTP[k] / (2*allTP[k] + allFP[k] + allFN[k])
        allF1M[k] = np.mean([F1d[x][k] for x in result_boxes])

    print('Best threshold: F1=', np.max(allF1), 'Threshold=',thresholds[np.argmax(allF1)])
        
    return thresholds[np.argmax(allF1)], allF1, thresholds


## Optimize threshold for TUPAC models


In [13]:
DBtupac = Database().open('TUPAC_AL/TUPAC_alternativeLabels_training.sqlite')
thresholds = {'TUPAC':{},'XR':{},'CS2':{},'S360':{}}
for run in np.arange(1,6):
    thr = optimize_threshold(DBtupac, result_boxes_train['TUPAC'][run])
    thresholds['TUPAC'][run]=thr[0]

Optimizing threshold for validation set of 73 files: 
Best threshold: F1= 0.7329224447868515 Threshold= 0.6400000000000003
Optimizing threshold for validation set of 73 files: 
Best threshold: F1= 0.7248391248391248 Threshold= 0.49000000000000016
Optimizing threshold for validation set of 73 files: 
Best threshold: F1= 0.7286470143613001 Threshold= 0.4300000000000001
Optimizing threshold for validation set of 73 files: 
Best threshold: F1= 0.7139622641509434 Threshold= 0.48000000000000015
Optimizing threshold for validation set of 73 files: 
Best threshold: F1= 0.7168792934249264 Threshold= 0.5300000000000002


In [14]:
DB = Database().open('MIDOG.sqlite')
#thresholds = {'TUPAC':{},'XR':{},'CS2':{},'S360':{}}
for scanner in ['XR','S360','CS2']:
    for run in np.arange(1,6):
        thr = optimize_threshold(DB, result_boxes_train[scanner][run])
        thresholds[scanner][run]=thr[0]

Optimizing threshold for validation set of 40 files: 
Best threshold: F1= 0.7003891050583657 Threshold= 0.4200000000000001
Optimizing threshold for validation set of 40 files: 
Best threshold: F1= 0.7576530612244898 Threshold= 0.5500000000000003
Optimizing threshold for validation set of 40 files: 
Best threshold: F1= 0.7428571428571429 Threshold= 0.5700000000000003
Optimizing threshold for validation set of 40 files: 
Best threshold: F1= 0.7564766839378239 Threshold= 0.6200000000000003
Optimizing threshold for validation set of 40 files: 
Best threshold: F1= 0.7107231920199502 Threshold= 0.5400000000000003
Optimizing threshold for validation set of 40 files: 
Best threshold: F1= 0.7628607277289837 Threshold= 0.5500000000000003
Optimizing threshold for validation set of 40 files: 
Best threshold: F1= 0.7865168539325843 Threshold= 0.6200000000000003
Optimizing threshold for validation set of 40 files: 
Best threshold: F1= 0.7888748419721872 Threshold= 0.46000000000000013
Optimizing thre

In [15]:
F1 = {'TUPAC': {}, 'XR': {}, 'S360':{}, 'CS2':{}}


In [16]:
result_boxes.keys()

dict_keys(['TUPAC', 'S360', 'CS2', 'XR'])

In [17]:
database = Database().open('MIDOG.sqlite')

XR={}
S360={}
CS2={}
F1
for run in np.arange(1,6):
    subset_XR = {f'{key:03d}.tiff': result_boxes['TUPAC'][run][f'{key:03d}.tiff'] for key in np.arange(41,51)}
    subset_S360 = {f'{key:03d}.tiff': result_boxes['TUPAC'][run][f'{key:03d}.tiff'] for key in np.arange(91,101)}
    subset_CS2 = {f'{key:03d}.tiff': result_boxes['TUPAC'][run][f'{key:03d}.tiff'] for key in np.arange(141,151)}
    XR[run] = calculate_F1(database,subset_XR, float(thresholds['TUPAC'][run]))
    S360[run] = calculate_F1(database,subset_S360, thresholds['TUPAC'][run])
    CS2[run] = calculate_F1(database,subset_CS2, thresholds['TUPAC'][run])
F1['TUPAC']['XR'] = XR
F1['TUPAC']['CS2'] = CS2
F1['TUPAC']['S360'] = S360



In [18]:
thresholds['TUPAC'][run]

0.5300000000000002

In [19]:
for scanner in ['XR','S360','CS2']:
    XR={}
    S360={}
    CS2={}
    for run in np.arange(1,6):
        subset_XR = {f'{key:03d}.tiff': result_boxes[scanner][run][f'{key:03d}.tiff'] for key in np.arange(41,51)}
        subset_S360 = {f'{key:03d}.tiff': result_boxes[scanner][run][f'{key:03d}.tiff'] for key in np.arange(91,101)}
        subset_CS2 = {f'{key:03d}.tiff': result_boxes[scanner][run][f'{key:03d}.tiff'] for key in np.arange(141,151)}
        XR[run] = calculate_F1(database,subset_XR, thresholds[scanner][run])
        S360[run] = calculate_F1(database,subset_S360, thresholds[scanner][run])
        CS2[run] = calculate_F1(database,subset_CS2, thresholds[scanner][run])
    F1[scanner]['XR'] = XR
    F1[scanner]['CS2'] = CS2
    F1[scanner]['S360'] = S360



In [20]:
for k in ['TUPAC','CS2','S360','XR']:
    for y in ['XR','CS2','S360']:
        print(k,'-',y,'->',np.mean(list(F1[k][y].values())))

TUPAC - XR -> 0.5530196039319556
TUPAC - CS2 -> 0.6126767264719035
TUPAC - S360 -> 0.40401938712468954
CS2 - XR -> 0.38978205617708167
CS2 - CS2 -> 0.7511055107990392
CS2 - S360 -> 0.4331682525015938
S360 - XR -> 0.4317875646331883
S360 - CS2 -> 0.5735624727734937
S360 - S360 -> 0.7206467988536656
XR - XR -> 0.5777512932058386
XR - CS2 -> 0.13761564427889653
XR - S360 -> 0.19049689805992326


In [21]:
for k in ['TUPAC','CS2','S360','XR']:
    for y in ['XR','CS2','S360']:
        print(k,'-',y,'->',np.std(list(F1[k][y].values())))

TUPAC - XR -> 0.03531442887465043
TUPAC - CS2 -> 0.04987030933795765
TUPAC - S360 -> 0.09017476055514971
CS2 - XR -> 0.09657761102397405
CS2 - CS2 -> 0.016233585938457865
CS2 - S360 -> 0.16632850178046749
S360 - XR -> 0.07742225497319043
S360 - CS2 -> 0.08670102366506721
S360 - S360 -> 0.025718316824473404
XR - XR -> 0.025306216752117708
XR - CS2 -> 0.13186975566551115
XR - S360 -> 0.058260316773786854


In [22]:
import pandas as pd

In [23]:
for k in ['TUPAC','XR','CS2','S360']:
    print(k,'& ',end='')
    inf_scanner=[]
    
    for y in ['XR','CS2','S360']:
        inf_scanner.append('%.2f $\\pm$ %.2f' % (np.mean(list(F1[k][y].values())), np.std(list(F1[k][y].values()))))
    print('&'.join(inf_scanner),'\\\\')

TUPAC & 0.55 $\pm$ 0.04&0.61 $\pm$ 0.05&0.40 $\pm$ 0.09 \\
XR & 0.58 $\pm$ 0.03&0.14 $\pm$ 0.13&0.19 $\pm$ 0.06 \\
CS2 & 0.39 $\pm$ 0.10&0.75 $\pm$ 0.02&0.43 $\pm$ 0.17 \\
S360 & 0.43 $\pm$ 0.08&0.57 $\pm$ 0.09&0.72 $\pm$ 0.03 \\


In [24]:
pickle.dump(F1, open('F1_MIDOG.p','wb'))

In [25]:
F1[k][y].values()

dict_values([0.74375, 0.6808510638297872, 0.7012987012987013, 0.7477744807121661, 0.7295597484276729])

One worker died.
